In [34]:
import os
import numpy as np
import pickle

from tensorflow.keras.models import Model
from sklearn.naive_bayes import GaussianNB
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array, load_img

In [30]:
# Data Directory
data_dir = 'medium_dataset'
mediums= ['Acrylic', 'Charcoal', 'Graphite', 'Oil', 'Pastel', 'Watercolor']

In [31]:
# Data Preprocessing and Feature Extraction
def extract_features(directory, model):
    features = []
    labels = []
    
    for medium in mediums:
        medium_dir = os.path.join(directory, medium)
        for img_name in os.listdir(medium_dir):
            img_path = os.path.join(medium_dir, img_name)
            img = load_img(img_path, target_size=(224, 224))
            img_array = img_to_array(img)
            img_array = np.expand_dims(img_array, axis=0)
            img_array = preprocess_input(img_array)
            
            feature = model.predict(img_array)
            features.append(feature.flatten())
            labels.append(medium)
    return np.array(features), np.array(labels)

In [16]:
# Load pre-trained VGG16 model + higher level layers
base_model = VGG16(weights='imagenet')
model = Model(inputs=base_model.input, outputs=base_model.get_layer('flatten').output)

In [17]:
# Extracting features
features, labels = extract_features(data_dir, model)

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 800ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 338ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 321ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 422ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 403ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 365ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 342ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 327ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 342ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 362ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 328ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 383ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 352ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 329ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 323ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 309ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 352ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 331ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 325ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 330ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 381ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 307ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 339ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 323ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 328ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 

In [18]:
# labels
label = LabelEncoder()
labels_encoded = label.fit_transform(labels)

In [19]:
# Model Training
X_train, X_test, y_train, y_test = train_test_split(features, labels_encoded, test_size=0.2, random_state=42)
nb_classifier = GaussianNB()
nb_classifier.fit(X_train, y_train)

GaussianNB()

In [22]:
# Model Evaluation
y_pred = nb_classifier.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred, target_names=label.classes_)

print(f'Accuracy: {accuracy}')
print('Classification Report:')
print(report)

Accuracy: 0.5641666666666667
Classification Report:
              precision    recall  f1-score   support

     Acrylic       0.47      0.47      0.47       215
    Charcoal       0.68      0.58      0.62       222
    Graphite       0.57      0.60      0.58       176
         Oil       0.64      0.78      0.70       204
      Pastel       0.44      0.40      0.42       189
  Watercolor       0.56      0.56      0.56       194

    accuracy                           0.56      1200
   macro avg       0.56      0.56      0.56      1200
weighted avg       0.56      0.56      0.56      1200



In [28]:
# Prediction on new data
def classify_image(img):
    img = img.resize((224, 224))
    img_array = img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = preprocess_input(img_array)
    
    feature = model.predict(img_array)
    feature = feature.flatten().reshape(1, -1)
    prediction = nb_classifier.predict(feature)
    medium = label.inverse_transform(prediction)
    return medium[0]

In [32]:
save_dir = 'saved_models'
os.makedirs(save_dir, exist_ok=True)

# Save the model and label encoder
with open(os.path.join(save_dir, 'medium_classifier.pkl'), 'wb') as medium_model:
    pickle.dump(nb_classifier, medium_model)

with open(os.path.join(save_dir, 'medium_label.pkl'), 'wb') as medium_label:
    pickle.dump(label, medium_label)